In [57]:
import pandas as pd

from collections import defaultdict
from comb_peak_files import comb_peak_files
from peak_weight import get_weight
from edge_list import edge_list_from_peak, edge_list_to_nx_graph
from feature_mat import get_all_feature_mat
from train_test import test_by_cv
from imp_score import imp_score
from coreg import get_coreg_module
from ConSReg import ConSReg

In [58]:
# ----------------------------------Edit the input file path here-------------------------------------------------------
dap_folder = "/work/alexsong/project/ConSReg/data/DAPseq/DAPseq_OMalley2016Cell/dap_data_v4/all_peaks/"
atac_file = "/work/alexsong/project/ConSReg/data/ATACseq/ATACseq_Lu2017NAR/peaks/root_merged.bed"
gff_file = "/work/alexsong/project/ConSReg/data/ATHRef/TAIR10_GFF3_genes.gff"
########################################################################################################################

In [59]:
sc_diff_file = "/work/alexsong/project/scConSReg/data/cortext-endodermis.csv"

In [60]:
analysis = ConSReg()

In [61]:
analysis.preprocess(dap_folder=dap_folder, diff_file=[sc_diff_file], atac_file=atac_file, gff_file=gff_file)

Merging DAP-seq peaks...
>> preparing features information...		 2018-09-21 09:42:41 PM 
>> identifying nearest features...		 2018-09-21 09:42:41 PM 
>> calculating distance from peak to TSS...	 2018-09-21 09:43:46 PM 
>> assigning genomic annotation...		 2018-09-21 09:43:46 PM 
>> assigning chromosome lengths			 2018-09-21 09:44:10 PM 
>> done...					 2018-09-21 09:44:10 PM 
Done
Overlapping DAP-seq with ATAC-seq...
Done
Reading diff tables...
Done


<ConSReg.ConSReg instance at 0x7f3e4ac20710>

In [62]:
analysis.gen_feature_mat()

Generating feature matrices...
Done


<ConSReg.ConSReg instance at 0x7f3e4ac20710>

In [65]:
analysis.feature_mat_list[0][1].shape

(1396, 311)

In [77]:
analysis.eval_by_cv(ml_engine = 'lrlasso')

Performing cross-validation for each feature matrix using lrlasso engine...
Done


<ConSReg.ConSReg instance at 0x7f3e4ac20710>

In [79]:
pd.DataFrame(analysis.auroc,columns=["up","down"]).to_csv("/work/alexsong/project/scConSReg/result/auroc_var_genes.csv")
pd.DataFrame(analysis.auprc,columns=["up","down"]).to_csv("/work/alexsong/project/scConSReg/result/auprc_var_genes.csv")

In [80]:
analysis.compute_imp_score(n_resampling = 200, n_jobs = 44, verbose = True)

Performing stability selection and compute importance score for each TF...
Done


<ConSReg.ConSReg instance at 0x7f3e4ac20710>

In [89]:
analysis.imp_score_list[0][0].sort_values(by="scores",ascending=False).to_csv("/work/alexsong/project/scConSReg/result/imp_score_up_09-21-18.csv")

In [90]:
analysis.imp_score_list[0][1].sort_values(by="scores",ascending=False).to_csv("/work/alexsong/project/scConSReg/result/imp_score_down_09-21-18.csv")